In [1]:
from lxml import html

from bs4 import BeautifulSoup
import time
import datetime

import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import NoSuchElementException

import json
import os

from apscheduler.schedulers.background import BackgroundScheduler
import sys

import helper

In [2]:
TIME_INTERVAL_SIGNIN_BOOKING = 180 #3 minutes
DELAY_TIME_BOOKING = 0.5

IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20

In [3]:
# NO_WINDOW = True

In [4]:
course_schedule = []

In [5]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element("xpath",xpath)

In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')
    
    send_keys(username, "DN20092360")
    send_keys(password, "jj1216")
#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))

    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()
    
def click_book_this_class_now(driver, wait):
    
    click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預定此課')]")
    
    # click_btn = driver.find_element("xpath","//a[contains(text(), '現在就預定此課')]") 
    # try:
    #     click_btn = driver.find_element("id",'ctl00_cphContents_btnBook')
    #     click_btn.click()
    # except NoSuchElementException:
    #     print("Error: click_book_this_class_now -- cannot find the book now button")
    
    print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False


In [7]:
def scheduled_sign_in(driver, wait, class_to_be_booked):
    print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    time.sleep(1)
    sign_in(driver, wait)

    
def reserve_class(driver, wait, class_to_be_booked):
    
    print("reserve_class: ({})\n".format(class_to_be_booked['time']))
    
    book_url = class_to_be_booked['href']
    class_name = class_to_be_booked['name']
    
    driver.get(book_url)

    click_book_this_class_now(driver, wait)

In [8]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [9]:
def create_new_driver():
    #start web driver

    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    service = Service(executable_path=r'/usr/bin/chromedriver')

    #setup option for chrome profile
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=service, options=options)

    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    

    return driver, wait

In [10]:
# load schedule from file
with open("./class_schedules/06 十一月 - 12 十一月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)

In [11]:
    # setup classes to be booked
    classes_to_be_booked = []

    ##---------------------------------------------------------------
    Day_string = "07"
    
    c = helper.search_courses_by_date(Day_string, course_schedule)
    c = helper.search_courses_by_name('', c)
    c = helper.search_courses_by_teacher('Mei', c)
    # c = helper.search_courses_by_time('pm', c)

    classes_to_be_booked.extend(c)
    
    c = helper.search_courses_by_date(Day_string, course_schedule)
    c = helper.search_courses_by_name('', c)
    c = helper.search_courses_by_teacher('Una', c)
    # c = helper.search_courses_by_time('pm', c)

    classes_to_be_booked.extend(c)
    ##---------------------------------------------------------------

    classes_to_be_booked

[{'room': '敦南 6 教室',
  'date': '07 11月 周二',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242701',
  'name': 'Hatha Yoga 哈達瑜珈',
  'teacher': 'Mei Wang',
  'time': '02:00 - 03:00 下午'},
 {'room': '敦南 9 教室',
  'date': '07 11月 周二',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242775',
  'name': 'Warm Stretch 溫和伸展',
  'teacher': 'Mei Wang',
  'time': '12:15 - 01:15 下午'},
 {'room': '敦南 4 教室',
  'date': '07 11月 周二',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242665',
  'name': 'Gentle Yoga 和緩瑜珈',
  'teacher': 'Una',
  'time': '04:00 - 05:00 下午'},
 {'room': '敦南 4 教室',
  'date': '07 11月 周二',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242666',
  'name': 'Vinyasa Intro 流暢瑜珈入門',
  'teacher': 'Una',
  'time': '06:00 - 07:00 下午'},
 {'room': '敦南 4 教室',
  'date': '07 11月 周二',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class

In [12]:
# retrieve class date and time, convert it in python's datetime structure then schedule a process to book!

drivers = []

scheduler = BackgroundScheduler()

for idx,class_to_be_booked in enumerate(classes_to_be_booked):

    #retrieve class date and time, convert it in python's datetime structure
    
    date_comp = class_to_be_booked['date'].split(" ")
    date_comp[1] = date_comp[1].replace("月","")
    date_comp.pop()
    month = int(date_comp[1])
    day = int(date_comp[0])
    # month,day

    time_comp = class_to_be_booked['time'].split(" ")
    hour = time_comp[0].split(":")[0]
    minute = time_comp[0].split(":")[1]
    hour = int(hour)
    minute = int(minute)

    if time_comp[3] == "下午":
        hour = hour+12

        # deal with pm notation but at noon 12 pm special case
        if hour == 24:
            hour = 12
    # hour,minute

    from datetime import date
    today = date.today()
    year = int(today.strftime("%Y"))
    class_datetime = datetime.datetime(year, month, day, hour, minute, 0)
    book_datetime = class_datetime - datetime.timedelta(seconds=72*60*60)
    print("book_datetime= {}".format(book_datetime))

    # setup web driver and scheduled process for each class

    driver, wait = create_new_driver()
    drivers.append(driver)

    now = datetime.datetime.now()

    if (book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)) < now:
        print("sign in and book immediately")
        
        signin_datetime = now + datetime.timedelta(seconds=4+idx)
        book_datetime = now + datetime.timedelta(seconds=10+idx)
        if book_datetime < now:
            print("too late for booking")
            continue

    else:
        signin_datetime = book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)
    
    book_datetime = book_datetime + datetime.timedelta(seconds=DELAY_TIME_BOOKING)
    
    print("current datetime: ", now)

    print("idx: ", idx, "scheduled signin datetime: ", signin_datetime)
    print("idx: ", idx, "scheduled book datetime: ", book_datetime)
    
    scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime, args=[driver, wait, class_to_be_booked])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, class_to_be_booked])

scheduler.start()
drivers

book_datetime= 2023-11-04 14:00:00
sign in and book immediately
current datetime:  2023-11-04 14:38:03.641430
idx:  0 scheduled signin datetime:  2023-11-04 14:38:07.641430
idx:  0 scheduled book datetime:  2023-11-04 14:38:14.141430
book_datetime= 2023-11-04 12:15:00
sign in and book immediately
current datetime:  2023-11-04 14:38:04.471048
idx:  1 scheduled signin datetime:  2023-11-04 14:38:09.471048
idx:  1 scheduled book datetime:  2023-11-04 14:38:15.971048
book_datetime= 2023-11-04 16:00:00
current datetime:  2023-11-04 14:38:05.300755
idx:  2 scheduled signin datetime:  2023-11-04 15:57:00
idx:  2 scheduled book datetime:  2023-11-04 16:00:00.500000
book_datetime= 2023-11-04 18:00:00
current datetime:  2023-11-04 14:38:06.124542
idx:  3 scheduled signin datetime:  2023-11-04 17:57:00
idx:  3 scheduled book datetime:  2023-11-04 18:00:00.500000
book_datetime= 2023-11-04 19:30:00
current datetime:  2023-11-04 14:38:06.951746
idx:  4 scheduled signin datetime:  2023-11-04 19:27:00

[<selenium.webdriver.chrome.webdriver.WebDriver (session="3b4c6a31556ad7e4f60a63c8342e0994")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="6a1ea62cbce5cea6dcf4ce981cd8d400")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="8f665b27457e14d94fdff1434ed6fd99")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="33b259bb98a14455717accae5ae5fb26")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="43eb4bb78c5b41a4c5925e9034fb95ec")>]

scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="3b4c6a31556ad7e4f60a63c8342e0994")>,<selenium.webdriver.support.wait.WebDriverWait (session="3b4c6a31556ad7e4f60a63c8342e0994")>)
scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="6a1ea62cbce5cea6dcf4ce981cd8d400")>,<selenium.webdriver.support.wait.WebDriverWait (session="6a1ea62cbce5cea6dcf4ce981cd8d400")>)
reserve_class: (02:00 - 03:00 下午)

click_btn:  <selenium.webdriver.remote.webelement.WebElement (session="3b4c6a31556ad7e4f60a63c8342e0994", element="72FA1382D2BC86FF4C6BB68F08F17491_element_22")>
reserve_class: (12:15 - 01:15 下午)

click_btn:  <selenium.webdriver.remote.webelement.WebElement (session="6a1ea62cbce5cea6dcf4ce981cd8d400", element="1CFBD02361559F99639EF89B632BF06D_element_20")>
